In [ ]:
from pyspark import SparkContext

# Inicialización del Contexto
sc = SparkContext.getOrCreate()
file_name = "calidad_aire_datos_meteo_mes.csv"

# Constantes
MAG_TEMP = 83
MAG_PREC = 89
FLAG_VALID = 'V'

# Carga del archivo
raw_rdd = sc.textFile(f"Sd_P1/{file_name}")

# Función para limpiar y estructurar una línea del CSV
def parse_line(line):
    try:
        parts = line.split(';')
        
        if len(parts) < 9: 
            return None
        
        # Extracción de datos
        provincia = parts[0]
        municipio = parts[1]
        estacion = parts[2]
        magnitud = int(parts[3])
        ano = parts[5]
        mes = parts[6]
        dia = parts[7]
        fecha = f"{ano}-{mes.zfill(2)}-{dia.zfill(2)}"
        
        valores_validos = []
        
        base_idx = 8
        for i in range(24):
            val_idx = base_idx + (i * 2)
            flag_idx = base_idx + (i * 2) + 1
            
            if flag_idx >= len(parts): 
                break
            
            val_str = parts[val_idx]
            flag = parts[flag_idx]
            
            if flag == FLAG_VALID:
                valor = float(val_str.replace(',', '.'))
                valores_validos.append(valor)
                
        return {
            'provincia':provincia,
            'fecha': fecha,
            'muni': municipio,
            'est': estacion,
            'mag': magnitud,
            'vals': valores_validos
        }
    except Exception:
        return None

data_rdd = raw_rdd.map(parse_line).filter(lambda x: x is not None)

# EJERCICIO 1:
temp_rdd = data_rdd.filter(lambda x: x['mag'] == MAG_TEMP)
temp_validos_rdd = temp_rdd.filter(lambda x: len(x['vals']) > 0)

print("--- EJERCICIO 1 ---")
print(f"Total registros de temperatura válidos: {temp_validos_rdd.count()}\n")

# EJERCICIO 2:
# Mappear por temperatura maxima por dias
daily_max_rdd = temp_validos_rdd.map(lambda x: (x['fecha'], max(x['vals'])))

# Reduccion para obtener el Max por fecha (ReduceByKey: Mantiene la tupla con el valor total más alto)
final_daily_max = daily_max_rdd.reduceByKey(lambda a, b: max(a, b)).sortByKey()

print("--- EJERCICIO 2: Máximos de Temperatura ---")
print("Fecha\t\tTemp_Max")
for date, t_max in final_daily_max.collect():
    print(f"{date}\t{t_max} ºC")
print("\n")


# EJERCICIO 3
# Reduccimos la tupla a partir de el valor de pa precipitacion (89)
precip_rdd = data_rdd.filter(lambda x: x['mag'] == MAG_PREC).map(lambda x: (x['fecha'], (x['muni'], x['est'], sum(x['vals']))))

max_precip_por_dia = precip_rdd.reduceByKey(lambda x, y: x if x[2] >= y[2] else y).sortByKey()

print("--- EJERCICIO 3A: Máximos de Lluvia por día ---")
# PARTE A: Imprimir las precipitaciones por municipios
for date, data in max_precip_por_dia.collect():
    print(f"Fecha: {date}, Muni: {data[0]}, Est: {data[1]}, Precip: {data[2]} mm")

# PARTE B: Máximo Absoluto del periodo 
max_absoluto = max_precip_por_dia.max(key=lambda x: x[1][2])

print("\n--- EJERCICIO 3B: Máximo Absoluto ---")
print(f"Fecha: {max_absoluto[0]}")
print(f"Municipio: {max_absoluto[1][0]}, Estación: {max_absoluto[1][1]}")
print(f"Precipitación Total: {max_absoluto[1][2]}\n")


# EJERCICIO 4
# Valor medio de las estaciones de municipios validos
def get_daily_avg(record):
    if len(record['vals']) == 0: return None
    avg = sum(record['vals']) / len(record['vals'])
    return (record['fecha'], avg)

# 1. Se crean RDDs separados para cada estación específica
ref_station = temp_rdd.filter(lambda x: int(x['muni']) == 6 and int(x['est']) == 4) \
                      .map(get_daily_avg).filter(lambda x: x is not None)

comp_station = temp_rdd.filter(lambda x: int(x['muni']) == 5 and int(x['est']) == 2) \
                       .map(get_daily_avg).filter(lambda x: x is not None)

# 2. Unimos los datos por Fecha
joined_data = ref_station.join(comp_station)

# 3. Calculamos el porcentaje
percentage_rdd = joined_data.mapValues(lambda x: (x[1] / x[0]) * 100).sortByKey()

print("--- EJERCICIO 4: Comparativa (Porcentaje) ---")
for date, pct in percentage_rdd.collect():
    print(f"Fecha: {date}, Porcentaje: {pct:.2f}%")

Total registros válidos: 1568
Fecha		Temp_Max
--- Máximos por día ---
Fecha: 2-01-0,0, Muni: 102, Est: 1, Precip: 0
Fecha: 2-02-0,0, Muni: 106, Est: 1, Precip: 0
Fecha: 2-02-0,2, Muni: 14, Est: 2, Precip: 0
Fecha: 2-02-0,3, Muni: 120, Est: 1, Precip: 0
Fecha: 2-02-0,4, Muni: 123, Est: 2, Precip: 0
Fecha: 2-02-0,7, Muni: 6, Est: 4, Precip: 0
Fecha: 2-02-0,8, Muni: 134, Est: 2, Precip: 0
Fecha: 2-02-1,0, Muni: 16, Est: 1, Precip: 0
Fecha: 2-02-1,1, Muni: 80, Est: 3, Precip: 0
Fecha: 2-02-1,2, Muni: 102, Est: 1, Precip: 0
Fecha: 2-02-1,8, Muni: 133, Est: 2, Precip: 0
Fecha: 2-02-2,2, Muni: 127, Est: 4, Precip: 0
Fecha: 2-02-3,0, Muni: 115, Est: 3, Precip: 0
Fecha: 2-03-0,0, Muni: 102, Est: 1, Precip: 0
Fecha: 2-03-0,1, Muni: 120, Est: 1, Precip: 0
Fecha: 2-03-0,2, Muni: 123, Est: 2, Precip: 0
Fecha: 2-03-0,4, Muni: 133, Est: 2, Precip: 0
Fecha: 2-03-1,0, Muni: 127, Est: 4, Precip: 0
Fecha: 2-04-0,6, Muni: 45, Est: 2, Precip: 0
Fecha: 2-04-0,7, Muni: 16, Est: 1, Precip: 0
Fecha: 2-04-0,8, 